# Build a RAG application with Milvus Lite, Mistral and Llama-index

In this notebook, we are showing how you can build a Retrieval Augmented Generation (RAG) application to interact with data from the French Parliament. It uses Ollama with Mistral for LLM operations, Llama-index for orchestration, and Milvus for vector storage.


## Install Ollama

In [ ]:
!curl https://ollama.ai/install.sh | sh

### Install the different dependencies 

In [6]:
!pip install -U pymilvus ollama llama-index-llms-ollama llama-index-vector-stores-milvus llama-index-readers-file llama-index-embeddings-mistralai

In [3]:
!pip install -U llama-index-embeddings-huggingface sentencepiece

  Using cached sentence_transformers-2.7.0-py3-none-any.whl.metadata (11 kB)
  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached scikit_learn-1.5.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scipy-1.13.1-cp311-cp311-macosx_12_0_arm64.whl.metadata (60 kB)
  Using cached sympy-1.12.1-py3-none-any.whl.metadata (12 kB)
  Using cached tokenizers-0.19.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached safetensors-0.4.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached sentence_transformers-2.7.0-py3-none-any.whl (171 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 MB 27.3 MB/s eta 0:00:00m eta 0:00:010:01:01
Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
Using cached scikit_learn-1.5.0-cp311-cp311-macosx_12_0_arm64.whl (11.0 MB)
Using cached scipy-1.13.1

### Download the Embedding model

Given that the text is written in French, I am using an embedding model that is specially trained on French Language, feel free to use a different one you can find on HuggingFace. 

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

MISTRAL_API_KEY = os.environ.get("MISTRAL_API_KEY")

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.mistralai import MistralAIEmbedding


model_name = "mistral-embed"
embed_model = MistralAIEmbedding(model_name=model_name, api_key=MISTRAL_API_KEY)

# embeddings = embed_model.get_text_embedding("La Plateforme - The Platform")

# embeddings = HuggingFaceEmbedding(model_name="dangvantuan/sentence-camembert-large")

/Users/stephen/Library/Caches/pypoetry/virtualenvs/fork-mistral-cookbook-9DpsfPiA-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Prepare out data to be stored in Milvus

This code makes it possible to process text embeddings using Sentence Camembert Large & Mistral-7B and store those in Milvus.

**!!Make sure to have Ollama running on your laptop!!**

* Initialises Mistral-7B model using Ollama
* Service Context: Configures a service context with Mistral and the embedding model defined above
* Vector Store: Sets up a collection in Milvus to store text embeddings, specifying the database file, collection name, vector dimensions
* Storage Context: Configures a storage context with the Milvus vector store

This makes it possible to have efficient storage and retrieval of vector embeddings for text data.

In [3]:
from llama_index.llms.ollama import Ollama
from llama_index.vector_stores.milvus import MilvusVectorStore

from llama_index.core import StorageContext, ServiceContext

llm = Ollama(model="mistral", request_timeout=120.0)

service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model, chunk_size=350)

vector_store = MilvusVectorStore(
    uri="milvus_mistral_rag.db",
    collection_name="mistral_french_parliament",
    dim=1024,  # the value changes with embedding model
    overwrite=True  # drop table if exist and then create
    
    )
storage_context = StorageContext.from_defaults(vector_store=vector_store)

/var/folders/kv/3dw9p_ts4b114chqt9m027pc0000gn/T/ipykernel_96753/2601041383.py:8: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model, chunk_size=350)


### Process and load the Data 

In [4]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

docs = SimpleDirectoryReader(input_files=['data/french_parliament_discussion.xml']).load_data()
vector_index = VectorStoreIndex.from_documents(docs, storage_context=storage_context, service_context=service_context)

In [5]:
from llama_index.core.tools import RetrieverTool, ToolMetadata

milvus_tool_openai = RetrieverTool(
    retriever=vector_index.as_retriever(similarity_top_k=3),  # retrieve top_k results
    metadata=ToolMetadata(
        name="CustomRetriever",
        description='Retrieve relevant information from provided documents.'
    ),
)

### Finally, ask questions to our RAG system

In [7]:
query_engine = vector_index.as_query_engine()
response = query_engine.query("What did the French parliament talk about the last time?")
print(response)

 The French parliament discussed disagreements regarding a text that was presented, with concerns being raised about it not addressing all demands made over the past two years and half. Propositions were made, evaluations requested, and suggestions such as systematically involving the Parliament and having the Scientific Council report its findings were expressed. However, no progress was made on these matters. Additionally, there were discussions about the end of the Assembly's self-censorship by the presidential majority and the decreasing number of laws compared to ordinances. A change in the assembly composition was seen as a hopeful sign for justifiable and strictly-encountered delegation of power to the Government. In a commission of laws, there had already been noted improvements with the current project law discussion.


In [ ]:
query_engine = vector_index.as_query_engine()
response = query_engine.query("Peux tu me dire de quoi à parler le parliement dernièrement? Je veux la réponse en Français")
print(response.response)

In [ ]:
!pip freeze > requirements.txt

---

#### If you like this tutorial, feel free to reach out on [LinkedIn](https://www.linkedin.com/in/stephen-batifol/), check out [Milvus](https://github.com/milvus-io/milvus) and join our [Discord](https://discord.gg/FG6hMJStWu).